# Example of using the PyRadiomics toolbox in Python

First, import some built-in Python modules needed to get our testing data.
Second, import the toolbox, only the `featureextractor` is needed, this module handles the interaction with other parts of the toolbox.

In [1]:
from __future__ import print_function
import six
import os  # needed navigate the system to get the input data

import radiomics
from radiomics import featureextractor  # This module is used for interaction with pyradiomics

### Setting up data

Before we can extract features, we need to get the input data, define the parameters for the extraction and instantiate the class contained within `featureextractor`.

In the next cell we get our testing data, this consists of an image and corresponding segmentation. This is also available from the PyRadiomics repository and is stored in `\pyradiomics\data`. The test case can be obtained using `radiomic.getTestCase`. This function returns the file location of the image and mask of the requested test case. If no locally available test case can be found, PyRadiomics downloads it from GitHub and stores it in temporary files. By specifying the (relative) path to the repository root, PyRadiomics is able to find test data included in the repository, preventing unnecessary downloads. If the this path is invalid (test case files not found), PyRadiomics will also revert to downloading it.

In [2]:
# Get the location of the repository, relative to this Notebook (assumes it is run from \pyradiomics\bin\Notebooks)
repositoryRoot = os.path.join(os.getcwd(), '..')
print('repositoryRoot, relative path:', repositoryRoot)
print('repositoryRoot, absolute path:', os.path.abspath(repositoryRoot))

# Get the testCase
imagePath, maskPath = radiomics.getTestCase('brain1', repositoryRoot)

if imagePath is None or maskPath is None:  # Something went wrong, in this case PyRadiomics will also log an error
    raise Exception('Error getting testcase!')  # Raise exception to prevent cells below from running in case of "run all"

# Additonally, store the location of the example parameter file, stored in \pyradiomics\examples/exampleSettings
paramPath = os.path.join(os.getcwd(), '..', 'examples', 'exampleSettings', 'Params.yaml')
print('Parameter file, absolute path:', os.path.abspath(paramPath))

repositoryRoot, relative path: E:\Git-Repos\pyradiomics\notebooks\..
repositoryRoot, absolute path: E:\Git-Repos\pyradiomics
Parameter file, absolute path: E:\Git-Repos\pyradiomics\examples\exampleSettings\Params.yaml


### Instantiating the extractor

Now that we have our input, we need to define the parameters and instantiate the extractor.
For this there are three possibilities:

1. Use defaults, don't define custom settings

2. Define parameters in a dictionary, control filters and features after initialisation

3. Use a parameter file

##### Method 1, use defaults

In [3]:
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeaturesExtractor()

print('Extraction parameters:\n\t', extractor.kwargs)
print('Enabled filters:\n\t', extractor.inputImages)
print('Enabled features:\n\t', extractor.enabledFeatures)

Extraction parameters:
	 {'distances': [1], 'additionalInfo': True, 'enableCExtensions': True, 'force2D': False, 'interpolator': 'sitkBSpline', 'resampledPixelSpacing': None, 'label': 1, 'normalizeScale': 1, 'normalize': False, 'force2Ddimension': 0, 'removeOutliers': None, 'minimumROISize': None, 'minimumROIDimensions': 1, 'padDistance': 5}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'firstorder': [], 'glcm': [], 'shape': [], 'glrlm': [], 'glszm': []}


##### Method 2, hard-coded settings:

In [4]:
# First define the parameters
params = {}
params['binWidth'] = 20
params['sigma'] = [1, 2, 3]
params['verbose'] = True

# Instantiate the extractor
extractor = featureextractor.RadiomicsFeaturesExtractor(**params)  # ** 'unpacks' the dictionary in the function call

print('Extraction parameters:\n\t', extractor.kwargs)
print('Enabled filters:\n\t', extractor.inputImages)  # Still the default settings
print('Enabled features:\n\t', extractor.enabledFeatures)  # Still the default settings

Extraction parameters:
	 {'distances': [1], 'verbose': True, 'additionalInfo': True, 'enableCExtensions': True, 'force2D': False, 'interpolator': 'sitkBSpline', 'resampledPixelSpacing': None, 'label': 1, 'normalizeScale': 1, 'normalize': False, 'force2Ddimension': 0, 'removeOutliers': None, 'minimumROISize': None, 'binWidth': 20, 'minimumROIDimensions': 1, 'sigma': [1, 2, 3], 'padDistance': 5}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'firstorder': [], 'glcm': [], 'shape': [], 'glrlm': [], 'glszm': []}


In [5]:
# This cell is equivalent to the previous cell
extractor = featureextractor.RadiomicsFeaturesExtractor(binWidth=20, sigma=[1, 2, 3], verbose=True)  # Equivalent of code above

print('Extraction parameters:\n\t', extractor.kwargs)
print('Enabled filters:\n\t', extractor.inputImages)  # Still the default settings
print('Enabled features:\n\t', extractor.enabledFeatures)  # Still the default settings

Extraction parameters:
	 {'distances': [1], 'verbose': True, 'additionalInfo': True, 'enableCExtensions': True, 'force2D': False, 'interpolator': 'sitkBSpline', 'resampledPixelSpacing': None, 'label': 1, 'normalizeScale': 1, 'normalize': False, 'force2Ddimension': 0, 'removeOutliers': None, 'minimumROISize': None, 'binWidth': 20, 'minimumROIDimensions': 1, 'sigma': [1, 2, 3], 'padDistance': 5}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'firstorder': [], 'glcm': [], 'shape': [], 'glrlm': [], 'glszm': []}


In [6]:
# Enable a filter (in addition to the 'Original' filter already enabled)
extractor.enableInputImageByName('LoG')
print('')
print('Enabled filters:\n\t', extractor.inputImages)

# Disable all feature classes, save firstorder
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('firstorder')
print('')
print('Enabled features:\n\t', extractor.enabledFeatures)

# Specify some additional features in the GLCM feature class
extractor.enableFeaturesByName(glcm=['Autocorrelation', 'Homogeneity1', 'SumSquares'])
print('')
print('Enabled features:\n\t', extractor.enabledFeatures)


Enabled filters:
	 {'Original': {}, 'LoG': {}}

Enabled features:
	 {'firstorder': []}

Enabled features:
	 {'firstorder': [], 'glcm': ['Autocorrelation', 'Homogeneity1', 'SumSquares']}


##### Method 3, using a parameter file

In [7]:
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeaturesExtractor(paramPath)

print('Extraction parameters:\n\t', extractor.kwargs)
print('Enabled filters:\n\t', extractor.inputImages)
print('Enabled features:\n\t', extractor.enabledFeatures)

Extraction parameters:
	 {'distances': [1], 'additionalInfo': True, 'enableCExtensions': True, 'weightingNorm': None, 'force2D': False, 'interpolator': 'sitkBSpline', 'resampledPixelSpacing': None, 'label': 1, 'normalizeScale': 1, 'normalize': False, 'force2Ddimension': 0, 'removeOutliers': None, 'minimumROISize': None, 'binWidth': 25, 'minimumROIDimensions': 1, 'padDistance': 5}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'firstorder': [], 'glcm': None, 'shape': ['Volume', 'SurfaceArea', 'SurfaceVolumeRatio', 'Sphericity', 'SphericalDisproportion', 'Maximum3DDiameter', 'Maximum2DDiameterSlice', 'Maximum2DDiameterColumn', 'Maximum2DDiameterRow', 'Elongation', 'Flatness'], 'glrlm': None, 'glszm': None}


### Extract features

Now that we have our extractor set up with the correct parameters, we can start extracting features:

In [8]:
result = extractor.execute(imagePath, maskPath)

In [9]:
print('Result type:', type(result))  # result is returned in a Python ordered dictionary)
print('')
print('Calculated features')
for key, value in six.iteritems(result):
    print('\t', key, ':', value)

Result type: <class 'collections.OrderedDict'>

Calculated features
	 general_info_BoundingBox : (162, 84, 11, 47, 70, 7)
	 general_info_GeneralSettings : {'distances': [1], 'additionalInfo': True, 'enableCExtensions': True, 'weightingNorm': None, 'force2D': False, 'interpolator': 'sitkBSpline', 'resampledPixelSpacing': None, 'label': 1, 'normalizeScale': 1, 'normalize': False, 'force2Ddimension': 0, 'removeOutliers': None, 'minimumROISize': None, 'binWidth': 25, 'minimumROIDimensions': 1, 'padDistance': 5}
	 general_info_ImageHash : 5c9ce3ca174f0f8324aa4d277e0fef82dc5ac566
	 general_info_ImageSpacing : (0.7812499999999999, 0.7812499999999999, 6.499999999999998)
	 general_info_InputImages : {'Original': {}}
	 general_info_MaskHash : 9dc2c3137b31fd872997d92c9a92d5178126d9d3
	 general_info_Version : 1.1.1.post75.dev0+g9db9f08
	 general_info_VolumeNum : 2
	 general_info_VoxelNum : 4137
	 original_shape_SphericalDisproportion : 2.06159321347
	 original_shape_Maximum2DDiameterSlice : 47.218